In [ ]:
""" 
3/17/21 - started before monthly meeting.
focus on across expt analyses.

"""


In [1]:
%load_ext autoreload
%autoreload 2

%cd ..

/data1/code/python/drawmonkey


In [6]:
from tools.utils import * 
from tools.plots import *
from tools.analy import *
from tools.calc import *
from tools.analyplot import *
from tools.preprocess import *
from tools.dayanalysis import *
from analysis.line2 import *
from analysis.modelexpt import *
from analysis.probedatTaskmodel import *

from pythonlib.drawmodel.analysis import *
from pythonlib.tools.stroketools import *

## COMPARE SAME TASK ACROSS EXPERIMENTS

In [ ]:
from analysis.probedatTaskmodel import *


## EXTRACT DATA ACROSS DAYS
for a in ["Pancho", "Red"]:
#     aelist = [
#         (a, "lines5"),
#         (a, "arc2")
#     ]
    aelist = [
        (a, "lines5redo"),
        (a, "lines5")
    ]

    DAT = []
    for ae in aelist:

        animal = ae[0]
        expt = ae[1]
        FD, MD = loadMultDataForExpt(expt, animal, whichdates="all", metadatonly=False)
        PROBEDAT = loadProbeDatWrapper(FD, MD, getnumstrokes=False)
        Probedat = ProbedatTaskmodel(PROBEDAT, MD)

        DAT.append(
            {"animal":animal,
             "expt":expt,
             "P":Probedat})



    ## Find common tasks across all expts
    names_list = []
    for D in DAT:
        P = D["P"]

        filt = {
    #         "traintest":["test"],
            "random_task":[False],
    #         "task_stagecategory":["linePlusLv2", "3linePlusL", "LplusL", "2linePlusL", "triangle"]
    #         "task_stagecategory":["triangle"]
    }

        Pfilt = P.filterProbedat(filt, modify_in_place=False)


        # only keep fixed tasks
        names_list.append(list(set(ProbedatTaskmodel(Pfilt, P.Metadat).pandas()["unique_task_name"])))





    ## EXTRACT TASKS PRESENT ACROSS EXPTS.
    shared_tasks = []
    for n in names_list[0]:
        if all([n in names for names in names_list]):
    #         print(n)
            shared_tasks.append(n)
    print(f"found {len(shared_tasks)} tasks common across Probedats")    

    ## MAKE SAVE DIR
    SDIR = "/data2/analyses/notebook/across_expt_comparison"

    suffix = f"{aelist[0][0]}-arc2_lines2"

    SDIRthis = f"{SDIR}/{suffix}"

    os.makedirs(SDIRthis, exist_ok=True)
    print(SDIRthis)


    ## ==== PLOT ONE GRID FOR EACH TASK
    for task in shared_tasks:
        # task = random.sample(shared_tasks, 1)[0]
        # "2linePlusL_1-savedset-1-26662"

        # extract this task across all data
        out = []

        for D in DAT:
        #     D["P"] = ProbedatTaskmodel(D["P"].Probedat, D["P"].Metadat)

            P = D["P"]
            Pp = D["P"].pandas()
            ct = 0

            # go thru each epoch
            epochlist = set(Pp["epoch"])
            for e in epochlist:

                # collect indices
                idxs = np.where((Pp["epoch"]==e) & (Pp["unique_task_name"]==task))[0]

                # collect fd and trial
                for i in idxs:

                    out.append({
                        "trial":P.t(i)["trial"],
                        "filedata":P.fd(i),
                        "expt-epoch":f"{P.Metadat['expt']}-{e}",
                        "count":ct
                    })

                    ct+=1


        #     out.append(D["P"].pandas()["unique_task_name"]==task


        cat1 = ["expt-epoch", set([o["expt-epoch"] for o in out])]
        cat2 = ["count", set([o["count"] for o in out])]



        fig = plotTrial2dGrid(out, cat1=cat1, cat2=cat2, plot_task_last_col=True, clean=True)


        fig.savefig(f"{SDIRthis}/2dgrid-{task}.pdf")
